In [17]:
import tensorflow as tf
import numpy as np
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2

class PlantDiseaseDetector:
    def __init__(self, data_dir='dataset', img_size=224):
        self.data_dir = Path(data_dir)
        self.img_size = img_size
        self.model = None
        self.history = None
        self.class_names = ['healthy', 'diseased']
    
    def setup_data_generators(self, batch_size=32):
        """Setup data generators with augmentation"""
        # Training data generator with augmentation
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            horizontal_flip=True,
            vertical_flip=False,
            fill_mode='nearest',
            zoom_range=0.15,
            brightness_range=[0.8, 1.2]
        )

        # Validation data generator (only rescaling)
        valid_datagen = ImageDataGenerator(rescale=1./255)

        # Create generators
        self.train_generator = train_datagen.flow_from_directory(
            self.data_dir / 'train',
            target_size=(self.img_size, self.img_size),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=True
        )

        self.valid_generator = valid_datagen.flow_from_directory(
            self.data_dir / 'validation',
            target_size=(self.img_size, self.img_size),
            batch_size=batch_size,
            class_mode='categorical',
            shuffle=False
        )

    def build_model(self):
        """Build the CNN model using ResNet50 as base"""
        # Load pre-trained ResNet50
        base_model = ResNet50(
            weights='imagenet',
            include_top=False,
            input_shape=(self.img_size, self.img_size, 3)
        )
        
        # Freeze the base model layers
        base_model.trainable = False
        
        # Create the model
        self.model = Sequential([
            base_model,
            GlobalAveragePooling2D(),
            Dense(1024, activation='relu'),
            Dropout(0.5),
            Dense(512, activation='relu'),
            Dropout(0.3),
            Dense(len(self.class_names), activation='softmax')
        ])
        
        # Compile the model
        self.model.compile(
            optimizer=Adam(learning_rate=0.001),
            loss='categorical_crossentropy',
            metrics=['accuracy', 'Precision', 'Recall']
        )

    def train(self, epochs=1, fine_tune=True):
        """Train the model with early stopping and learning rate reduction"""
        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=5,
                restore_best_weights=True
            ),
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.2,
                patience=3,
                min_lr=1e-6
            )
        ]

        # Initial training
        print("Initial training...")
        self.history = self.model.fit(
            self.train_generator,
            epochs=epochs,
            validation_data=self.valid_generator,
            callbacks=callbacks
        )

        if fine_tune:
            print("\nFine-tuning the model...")
            # Unfreeze the last few layers of the base model
            self.model.layers[0].trainable = True
            for layer in self.model.layers[0].layers[-20:]:
                layer.trainable = True

            # Recompile with lower learning rate
            self.model.compile(
                optimizer=Adam(learning_rate=1e-5),
                loss='categorical_crossentropy',
                metrics=['accuracy', 'Precision', 'Recall']
            )

            # Fine-tune
            history_fine = self.model.fit(
                self.train_generator,
                epochs=10,
                validation_data=self.valid_generator,
                callbacks=callbacks
            )

            # Combine histories
            for k in self.history.history:
                self.history.history[k].extend(history_fine.history[k])

    def plot_training_history(self):
        """Plot training history including accuracy and loss"""
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

        # Accuracy plot
        ax1.plot(self.history.history['accuracy'])
        ax1.plot(self.history.history['val_accuracy'])
        ax1.set_title('Model Accuracy')
        ax1.set_ylabel('Accuracy')
        ax1.set_xlabel('Epoch')
        ax1.legend(['Train', 'Validation'])

        # Loss plot
        ax2.plot(self.history.history['loss'])
        ax2.plot(self.history.history['val_loss'])
        ax2.set_title('Model Loss')
        ax2.set_ylabel('Loss')
        ax2.set_xlabel('Epoch')
        ax2.legend(['Train', 'Validation'])

        plt.show()

    def save_model(self, filepath='high_plant_disease_model.h5'):
        """Save the trained model"""
        self.model.save(filepath)
        print(f"Model saved to {filepath}")

    def load_saved_model(self, filepath='high_plant_disease_model.h5'):
        """Load a saved model"""
        self.model = load_model(filepath)
        print("Model loaded successfully!")

    def preprocess_image(self, image_path):
        """Preprocess a single image for prediction"""
        # Read and resize image
        img = cv2.imread(str(image_path))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (self.img_size, self.img_size))
        
        # Preprocess
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)
        
        return img

    def predict(self, image_path):
        """Predict disease status for a single image"""
        # Preprocess the image
        img = self.preprocess_image(image_path)
        
        # Make prediction
        prediction = self.model.predict(img)
        predicted_class = self.class_names[np.argmax(prediction[0])]
        confidence = np.max(prediction[0]) * 100
        
        return {
            'status': predicted_class,
            'confidence': confidence,
            'probabilities': {
                class_name: float(prob) * 100 
                for class_name, prob in zip(self.class_names, prediction[0])
            }
        }

# Example usage
if __name__ == "__main__":
    # Initialize detector
    detector = PlantDiseaseDetector()
    
    # Setup and train
    detector.setup_data_generators()
    detector.build_model()
    detector.train(epochs=2)
    
    # Plot results and save model

    detector.plot_training_history()
    detector.save_model()
    
    
    # Example prediction
    result = detector.predict('bacteria.jpg')
    print(f"Plant Status: {result['status']}")
    print(f"Confidence: {result['confidence']:.2f}%")
    print("\nProbabilities:")
    for class_name, prob in result['probabilities'].items():
        print(f"{class_name}: {prob:.2f}%")

Found 14445 images belonging to 2 classes.
Found 3095 images belonging to 2 classes.
Initial training...
Epoch 1/2
452/452 ━━━━━━━━━━━━━━━━━━━━ 1652s 4s/step - Precision: 0.8350 - Recall: 0.8350 - accuracy: 0.8350 - loss: 0.5061 - val_Precision: 0.8439 - val_Recall: 0.8439 - val_accuracy: 0.8439 - val_loss: 0.4239 - learning_rate: 0.0010
Epoch 2/2
452/452 ━━━━━━━━━━━━━━━━━━━━ 2010s 4s/step - Precision: 0.8493 - Recall: 0.8493 - accuracy: 0.8493 - loss: 0.4251 - val_Precision: 0.8439 - val_Recall: 0.8439 - val_accuracy: 0.8439 - val_loss: 0.4416 - learning_rate: 0.0010

Fine-tuning the model...
Epoch 1/10
 14/452 ━━━━━━━━━━━━━━━━━━━━ 3:46:50 31s/step - Precision: 0.5729 - Recall: 0.5729 - accuracy: 0.5729 - loss: 0.7199

KeyboardInterrupt: 

In [25]:
# ... existing code ...

if __name__ == "__main__":
    # Initialize detector
    detector = PlantDiseaseDetector()
    
    # Load the saved model
    detector.load_saved_model('plant_disease_model.h5')
    
    # Example predictions
    image_paths = ['5.jpg']  # Add your image paths here
    
    for image_path in image_paths:
        result = detector.predict(image_path)
        print(f"\nResults for {image_path}:")
        print(f"Plant Status: {result['status']}")
        print(f"Confidence: {result['confidence']:.2f}%")
        print("Probabilities:")
        for class_name, prob in result['probabilities'].items():
            print(f"{class_name}: {prob:.2f}%")

# ... existing code ...

Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 917ms/step

Results for 5.jpg:
Plant Status: diseased
Confidence: 91.58%
Probabilities:
healthy: 6.36%
diseased: 91.58%
